# EasyEdit Example with T5 and KN

The following example shows how to use KN edit with T5.

## Preparation

Except for installing all the necessary dependencies, you need to download the pre-trained language model. The default hparams in the 'hparams' folder use huggingface cache in default. You can change the 'model_name' in hparam file to the model name to let huggingface automatically download the model for you. In our example, it means changing the 'model_name' to t5-3B.

In [ ]:
from easyeditor import KNHyperParams
from easyeditor import BaseEditor
import os

hparams = KNHyperParams.from_hparams("Your_Path_To_Repo/EasyEdit/hparams/KN/t5-3B.yaml")

prompts = ['Who was the designer of Lahti Town Hall?']
target_new = ['Joe Biden']
subject = ['Lahti Town Hall']

editor=BaseEditor.from_hparams(hparams)
metrics, edited_model, _ = editor.edit(
    prompts=prompts,
    ground_truth=None,
    target_new=target_new,
    subject=subject,
    keep_original_weight=False
)
print(metrics)
print(type(edited_model))

## Output

```
Getting coarse neurons for each prompt...: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:02<00:00, 62.81s/it]

61 coarse neurons found - refining
61 neurons remaining after refining

Before modification - groundtruth probability: tensor([2.5644e-29], device='cuda:4')
Argmax completion: `</s></s><extra_id_5>`
Argmax prob: 0.21097961723932296

After modification - groundtruth probability: tensor([4.9062e-22], device='cuda:4')
Argmax completion: `dendendenden`
Argmax prob: 0.9392140688144989
2023-11-13 20:50:04,281 - easyeditor.editors.editor - INFO - Execution 0 editing took 63.00693202018738
11/13/2023 20:50:04 - INFO - easyeditor.editors.editor -   Execution 0 editing took 63.00693202018738
2023-11-13 20:50:04,359 - easyeditor.editors.editor - INFO - Evaluation took 0.07802677154541016
11/13/2023 20:50:04 - INFO - easyeditor.editors.editor -   Evaluation took 0.07802677154541016
2023-11-13 20:50:04,359 - easyeditor.editors.editor - INFO - 0 editing: Who was the designer of Lahti Town Hall? -> Joe Biden  
 {'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Who was the designer of Lahti Town Hall?', 'target_new': 'Joe Biden', 'ground_truth': '<|endoftext|>', 'portability': {}, 'locality': {}, 'subject': 'Lahti Town Hall'}, 'time': 63.00693202018738, 'post': {'rewrite_acc': [0.3333333432674408], 'locality': {}, 'portability': {}}}
11/13/2023 20:50:04 - INFO - easyeditor.editors.editor -   0 editing: Who was the designer of Lahti Town Hall? -> Joe Biden  
 {'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Who was the designer of Lahti Town Hall?', 'target_new': 'Joe Biden', 'ground_truth': '<|endoftext|>', 'portability': {}, 'locality': {}, 'subject': 'Lahti Town Hall'}, 'time': 63.00693202018738, 'post': {'rewrite_acc': [0.3333333432674408], 'locality': {}, 'portability': {}}}
[{'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Who was the designer of Lahti Town Hall?', 'target_new': 'Joe Biden', 'ground_truth': '<|endoftext|>', 'portability': {}, 'locality': {}, 'subject': 'Lahti Town Hall'}, 'time': 63.00693202018738, 'post': {'rewrite_acc': [0.3333333432674408], 'locality': {}, 'portability': {}}}]
<class 'transformers.models.t5.modeling_t5.T5ForConditionalGeneration'>
```

## Compare edited model and original model

In [ ]:
from transformers import T5ForConditionalGeneration
from transformers import T5Tokenizer
device = 'cuda'

tokenizer = T5Tokenizer.from_pretrained("t5-3B")
origin_model = T5ForConditionalGeneration.from_pretrained("t5-3B").to(device)


In [ ]:
batch = tokenizer(prompts, return_tensors='pt', padding=True, max_length=30).to(device)

origin_output = origin_model.generate(
    input_ids=batch['input_ids'].to(device),
    attention_mask=batch['attention_mask'].to(device),
    max_length=15
)

output = edited_model.generate(
    input_ids=batch['input_ids'].to(device),
    attention_mask=batch['attention_mask'].to(device),
    max_length=15
)

print(tokenizer.batch_decode(origin_output, skip_special_tokens=True))
print(tokenizer.batch_decode(output, skip_special_tokens=True))